# Capstone Project Notebook

##### "In this assignment, you will be required to explore, segment, and cluster the neighborhoods in the city of Toronto. However, unlike New York, the neighborhood data is not readily available on the internet. What is interesting about the field of data science is that each project can be challenging in its unique way, so you need to learn to be agile and refine the skill to learn new libraries and tools quickly depending on the project.

##### For the Toronto neighborhood data, a Wikipedia page exists that has all the information we need to explore and cluster the neighborhoods in Toronto. You will be required to scrape the Wikipedia page and wrangle the data, clean it, and then read it into a pandas dataframe so that it is in a structured format like the New York dataset.

##### Once the data is in a structured format, you can replicate the analysis that we did to the New York City dataset to explore and cluster the neighborhoods in the city of Toronto."

In [1]:
# import preliminary libraries etc
import numpy as np 
import pandas as pd
import json
import seaborn as sns

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
print('Hello Capstone Project Course!')

Hello Capstone Project Course!


### Leg 1: Segmenting and Clustering Neighborhoods
#### Obtain data from Wikipedia website

In [3]:
# Ended up not using Beautiful soup

#!pip install BeautifulSoup4
#!pip install requests
#from bs4 import BeautifulSoup
#import requests
#from urllib.request import urlopen
#!pip install lxml

In [ ]:
# install to read from wiki
!pip install wikipedia

In [4]:
import wikipedia as wp

In [5]:
# Obtain the page information
html = wp.page("List of postal codes of Canada: M").html().encode("UTF-8")
df_tor = pd.read_html(html)[0]

#Delete rows where borough is not assigned
df_tor = df_tor[df_tor['Borough'] != 'Not assigned']

# change column names
column_names = ['PostalCode','Borough', 'Neighborhood'] 
df_tor.columns = column_names
df_tor.head(2)

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village


In [6]:
#Combine all neighborhoods where postcode and borough are the same
df_tor = df_tor.groupby(['PostalCode', 'Borough'])['Neighborhood'].apply(list).apply(lambda x:', '.join(x)).to_frame().reset_index()

# reset the index
df_tor.reset_index()

# view the table, viewing more rows shows distinct postal codes
df_tor.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [7]:
# inspect size of dataset
df_tor.shape

(103, 3)

In [15]:
import io
import requests

# add in longitude and latitude of postal codes
url="https://cocl.us/Geospatial_data"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))

# join location parameters to original dataset
df_torl = df_tor.join(c.set_index('Postal Code'), on='PostalCode')
df_torl.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [10]:
from geopy.geocoders import Nominatim

In [11]:
address = 'Toronto'
# initialize variables to create folium map
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [13]:
import folium

In [17]:
# create map of New York using latitude and longitude values
map_tor = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_torl['Latitude'], df_torl['Longitude'], df_torl['Borough'], df_torl['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_tor)  
# show toronto map overlayed with neighborhoods
map_tor